# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import os

# Funciones

In [249]:
def get_hours(x,y):
    val = y.replace(' ','')
    if val[-1]=='h':
        return float(val[:-1])
    elif val.isdigit():
        return float(val)
    
    val = x.replace(' ','')
    if val[-1]=='h':
        return float(val[:-1])
    elif val.isdigit():
        return float(val)
    else:
        return np.nan

# Ruta de archivos

In [13]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_etl = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))

# Costos de cursos

# Funciones

In [40]:
def set_name_area(x):
    if x in ['administracion','administracion y negocios','negocios y administracion','administración y legislación\n','empresariales','gestion empresarial','facultad de cienciasadministrativas','faculta de jurisprudencia ,  ciencias']:
        return 'administración y legislación'
    elif x in ['salud','facultad ciencias ce la salud']:
        return 'actividades de salud'
    elif x in ['tecnologicos','tecnologia','tecnologia e innovacion','tecnologia e informacion','tics','tecnología de la información y comunicación',' tecnología de la información y comunicación']:
        return 'tecnologías de la información y comunicación'
    elif x in ['educativos','educacion','facultad de ciencias de la educacion']:
        return 'educación y capacitación'
    elif x in ['mecanica automotriz']:
        return 'mecánica automotriz'
    elif x in ['social']:
        return 'trabajo social'
    elif x in ['financiero','finanzas, comercio y ventas']:
        return 'servicios financieros'
    elif x in ['facultad  de ciencias agropecuarias']:
        return 'industria agropecuaria'
    elif x in ['agronomo']:
        return 'agronomía'
    elif x in ['comunicación']:
        return 'comunicación y artes gráficas'
    elif x in ['arte']:
        return 'artes y artesanía'
    else:
        return 'area base'
    

In [106]:
def set_name_modalidad(x):
    if 'online' in x or 'on line' in x:
        return 'online'
    if 'presencial' in x:
        return 'presencial'
    if 'virtual' in x or 'en vivo' in x:
        return 'virtual'
    else:
        return x
    

In [78]:
def set_precio(x):
    x = x.replace('$','')
    x = x.replace(',','')
    x = x.replace(' ','')
    x = x.replace('-','')
    
    if 'llamar' in x:
        return np.nan
    if ''==x:
        return np.nan
    return float(x)
    

In [283]:
def fill_precio_na(x,y):   
    if pd.isna(x['precio_set']):
        if x['modalidad_set'] in y['mean_by_mod'].keys():
            x['mean_by_mod'] = y['mean_by_mod'][x['modalidad_set']]
            
        if x['area_familia'] in y['mean_by_area'].keys():
            x['mean_by_area'] =  y['mean_by_area'][x['area_familia']]
            
        if x['Ciudad'] in y['mean_by_ciudad'].keys():
            x['mean_by_ciudad'] =  y['mean_by_ciudad'][x['Ciudad']]
            
            
        if x['modalidad_set']+'|'+x['area_familia'] in y['mean_by_ma'].keys():
            x['mean_by_ma'] =  y['mean_by_ma'][x['modalidad_set']+'|'+x['area_familia']]
            
        if x['area_familia']+'|'+x['Ciudad'] in y['mean_by_ac'].keys():
            x['mean_by_ac'] =  y['mean_by_ac'][x['area_familia']+'|'+x['Ciudad']]
            
        if x['modalidad_set']+'|'+x['Ciudad'] in y['mean_by_mc'].keys():
            x['mean_by_mc'] =  y['mean_by_mc'][x['modalidad_set']+'|'+x['Ciudad']]
            
            
        #if x['area_familia']+'|'+x['modalidad_set']+'|'+x['Ciudad'] in y['mean_by_mac'].keys():
        #    x['mean_by_mac'] =  y['mean_by_mac'][x['area_familia']+'|'+x['modalidad_set']+'|'+x['Ciudad']]
    return x

In [269]:
mean_by_mod = {}
mean_by_area = {}
mean_by_ciudad = {}
mean_by_ma = {}
mean_by_ac = {}
mean_by_mc = {}
mean_by_mac = {}
mean_list = {'mean_by_mod':mean_by_mod,'mean_by_area':mean_by_area,'mean_by_ciudad':mean_by_ciudad,'mean_by_ma':mean_by_ma,'mean_by_ac':mean_by_ac,'mean_by_mc':mean_by_mc}
for val in df_coec['modalidad_set'].value_counts().index.array:
    mean_by_mod[val]=df_coec[df_coec['modalidad_set']==val]['precio_set'].mean()
    for val2 in df_coec['area_familia'].value_counts().index.array:
        mean_by_ma[val+'|'+val2]=df_coec[(df_coec['area_familia']==val2)&(df_coec['modalidad_set']==val)]['precio_set'].mean()
        for val3 in df_coec['Ciudad'].value_counts().index.array:
            mean_by_mac[val+'|'+val2+'|'+val3]=df_coec[(df_coec['area_familia']==val2)&(df_coec['modalidad_set']==val)&(df_coec['Ciudad']==val3)]['precio_set'].mean()
        
        
for val in df_coec['area_familia'].value_counts().index.array:
    mean_by_area[val]=df_coec[df_coec['area_familia']==val]['precio_set'].mean()
    for val2 in df_coec['Ciudad'].value_counts().index.array:
        mean_by_ac[val+'|'+val2]=df_coec[(df_coec['Ciudad']==val2)&(df_coec['area_familia']==val)]['precio_set'].mean()
        
for val in df_coec['Ciudad'].value_counts().index.array:
    mean_by_ciudad[val]=df_coec[df_coec['Ciudad']==val]['precio_set'].mean()
    for val2 in df_coec['modalidad_set'].value_counts().index.array:
        mean_by_mc[val2+'|'+val]=df_coec[(df_coec['modalidad_set']==val2)&(df_coec['Ciudad']==val)]['precio_set'].mean()

In [278]:
# Selecting Data
df_coec = pd.read_csv(os.path.join(path_additional_data,'costo_oferta_educacion_continua.csv'))
df_coec = df_coec[['Ciudad','Areas de Formación', 'Nombre  de los Programas','Modalidad', 'Tiempo de duración','Duración programa',' Precio ']]
df_coec = df_coec.rename(columns={' Precio ':'Precio'})
for col in df_coec.columns:
    df_coec[col] = df_coec[col].astype('str').str.lower()
    aux = df_coec[col][0]
    for i in range(df_coec.shape[0]):
        if pd.isna(df_coec[col][i]) or df_coec[col][i]=='nan' :
            df_coec[col][i] = aux
        else:
            aux = df_coec[col][i]
            


In [279]:
df_coec['area_familia'] = df_coec.apply(lambda x: set_name_area(x['Areas de Formación']), axis=1 )

In [280]:
df_coec['modalidad_set'] = df_coec.apply(lambda x: set_name_modalidad(x['Modalidad']), axis=1 )

In [281]:
df_coec['precio_set'] = df_coec.apply(lambda x: set_precio(x['Precio']), axis=1 )

In [284]:
df_coec = df_coec.apply(lambda x: fill_precio_na(x,mean_list), axis=1)

In [285]:
df_coec['carga_horaria'] = df_coec.apply(lambda x: get_hours(x['Tiempo de duración'], x['Duración programa']), axis=1)

In [297]:
df_coec['carga_horaria'] = df_coec['carga_horaria'].fillna(df_coec['carga_horaria'].mean())

In [298]:
df_coec['precio_por_hora'] = df_coec.apply(lambda x: np.round(x['precio_set']/x['carga_horaria'],decimals = 2), axis=1)
df_coec['precio_por_hora'] = df_coec['precio_por_hora'].fillna(df_coec['precio_por_hora'].mean())

In [308]:
def get_prices_by(x,y):
    for mean_by in y.keys():
        x['precio_'+mean_by] =  np.round(x[mean_by]/x['carga_horaria'],decimals = 2)
    return x

In [309]:
df_coec = df_coec.apply(lambda x: get_prices_by(x,mean_list), axis=1)

In [315]:
df_coec_final = df_coec[['Ciudad','area_familia','modalidad_set','carga_horaria','precio_por_hora','precio_mean_by_mod','precio_mean_by_area','precio_mean_by_ciudad',	'precio_mean_by_ma','precio_mean_by_ac','precio_mean_by_mc']]

In [316]:
df_coec_final = df_coec_final.rename(columns={'Ciudad':'ciudad'})

In [317]:
df_coec_final.head(10)

,ciudad,area_familia,modalidad_set,carga_horaria,precio_por_hora,precio_mean_by_mod,precio_mean_by_area,precio_mean_by_ciudad,precio_mean_by_ma,precio_mean_by_ac,precio_mean_by_mc
0,quito,administración y legislación,online,1736.0,6.875873,0.10,0.21,0.1,0.07,0.06,0.18
1,quito,educación y capacitación,online,1680.0,6.875873,0.10,0.08,0.1,0.12,0.05,0.18
2,quito,actividades de salud,online,1680.0,6.875873,0.10,0.31,0.1,0.04,0.54,0.18
3,quito,administración y legislación,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
4,quito,administración y legislación,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
5,quito,administración y legislación,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
6,quito,administración y legislación,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
7,quito,administración y legislación,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
8,quito,administración y legislación,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
9,quito,tecnologías de la información y comunicación,online,168.0,6.875873,1.01,1.91,1.0,1.59,1.25,1.84


In [7]:
df_coec.to_csv('costos_cursos_analisis.csv',index=False)

In [25]:
df_coec = df_coec.drop(['Areas de Formación','Tiempo de duración','Duración programa'],axis=1)

In [27]:
df_coec['carga_horaria'] = df_coec['carga_horaria'].astype('int32')
df_coec['precio'] = df_coec.pop('Precio')

In [28]:
df_coec['precio_por_hora'] = df_coec.apply(lambda x: np.round(x['precio']/x['carga_horaria'],decimals = 2), axis=1)

In [29]:
df_coec['Modalidad'] = df_coec['Modalidad'].replace({'en vivo':'online','virtual':'online'})

In [30]:
df_coec = df_coec.rename(columns={'Nombre  de los Programas':'curso','Modalidad':'modalidad'})

In [32]:
df_coec.to_csv(os.path.join(path_additional_data,'costos_cursos_educacion_continua_tec.csv'),index=False)

Trabajar manualmente los nombres de costos_cursos_educacion_continua_tec.csv

In [33]:
df_coec = pd.read_csv(os.path.join(path_additional_data,'costos_cursos_educacion_continua_tec_opt.csv'))


In [34]:
df_ccce = pd.read_csv(os.path.join(path_additional_data,'costos_de_cursos_cec_epn.csv'))
df_ccce = df_ccce.apply(lambda x: x.astype(str).str.lower())

In [37]:
df_costos_cursos = pd.concat([df_coec,df_ccce],axis=0)

In [38]:
df_costos_cursos['carga_horaria']=df_costos_cursos['carga_horaria'].astype('int64')
df_costos_cursos['precio']=df_costos_cursos['precio'].astype('float64')
df_costos_cursos['precio_por_hora']=df_costos_cursos['precio_por_hora'].astype('float64')

In [42]:
#df_costos_cursos.head()
#df_costos_cursos.info()

In [43]:
df_costos_cursos.to_csv(os.path.join(path_etl,'costos_estimados_cursos.csv'),index=False)

# Datos de Ubicación

In [45]:
df_ubicacion = pd.read_csv(os.path.join(path_additional_data,'Ubicacion.csv'))
df_ubicacion = df_ubicacion.apply(lambda x: x.astype(str).str.lower())
df_ubicacion = df_ubicacion.drop_duplicates()

In [9]:
#df_ubicacion.head()
#df_ubicacion.info()

In [46]:
df_ubicacion.to_csv(os.path.join(path_etl,'ubicacion.csv'),index=False)